In [1]:
import pandas as pd


## Pre process


Start of with extracting the lowest observed heartrate during sleep for userid over all days recored.

In [2]:
heart_rate_data = pd.read_csv('data/heartrate_seconds_merged.csv')

sleep_data = pd.read_csv('data/minuteSleep_merged.csv')

heart_rate_df = pd.DataFrame(heart_rate_data)

sleep_df = pd.DataFrame(sleep_data)

heart_rate_df['Time'] = pd.to_datetime(heart_rate_df['Time'])
sleep_df['date'] = pd.to_datetime(sleep_df['date']).dt.date  # Extracting only date

When you merge the heart rate data with the sleep data based on the user ID and date, each heart rate measurement will be matched with the sleep data point that falls within the same minute. This ensures that you're considering the heart rate data closest in time to each sleep data point, even though the sampling frequencies are different.

In [3]:
# Merge heart rate and sleep data based on user ID and date
merged_df = pd.merge(heart_rate_df, sleep_df, left_on=['Id', heart_rate_df['Time'].dt.date], right_on=['Id', 'date'])

In [5]:
# Find the lowest observed heart rate during sleep for each user and date
resting_heart_rate_df = merged_df.groupby(['Id', 'date'])['Value'].min().reset_index()

# Rename columns for clarity
resting_heart_rate_df.columns = ['Id', 'Date', 'Resting Heart Rate']



In [8]:
resting_heart_rate_df[resting_heart_rate_df['Id'] == 2026352035]

,Id,Date,Resting Heart Rate
0,2026352035,2016-04-02,57
